In [10]:
import json
import pennylane as qml
import pennylane.numpy as np

def create_Hamiltonian(h):
    """
    Function in charge of generating the Hamiltonian of the statement.

    Args:
        h (float): magnetic field strength

    Returns:
        (qml.Hamiltonian): Hamiltonian of the statement associated to h
    """


    # Put your code here #
    coeff = [-1, -1, -1, -1, -h, -h, -h, -h]

    obs = [qml.PauliZ(0)@qml.PauliZ(1), qml.PauliZ(1)@qml.PauliZ(2), qml.PauliZ(2)@qml.PauliZ(3), qml.PauliZ(3)@qml.PauliZ(1), qml.PauliX(0), qml.PauliX(1), qml.PauliX(2), qml.PauliX(3)]

    H = qml.Hamiltonian(coeff,obs)
    return H

dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def model(params, H):
    """
    To implement VQE you need an ansatz for the candidate ground state!
    Define here the VQE ansatz in terms of some parameters (params) that
    create the candidate ground state. These parameters will
    be optimized later.

    Args:
        params (numpy.array): parameters to be used in the variational circuit
        H (qml.Hamiltonian): Hamiltonian used to calculate the expected value

    Returns:
        (float): Expected value with respect to the Hamiltonian H
    """


    # Put your code here #    
    def circuit(params):
        # Put your code here #    
        qml.RZ(params[0], 0)
        qml.RZ(params[1], 1)
        qml.RZ(params[2], 2)
        qml.RY(params[0], 0)
        qml.RY(params[1], 1)
        qml.RY(params[2], 2)
        for i in range(3):
            qml.CNOT(wires = [0,i+1])
        for i in range(4):
            qml.RY(params[3],i)
    circuit(params)
    return qml.expval(H)



def train(h):
    """
    In this function you must design a subroutine that returns the
    parameters that best approximate the ground state.

    Args:
        h (float): magnetic field strength

    Returns:
        (numpy.array): parameters that best approximate the ground state.
    """


    # Put your code here #
    H = create_Hamiltonian(h)
    max_iterations = 1000
    conv_tol = 1e-06
    step_size = 0.01
    init_arr = np.random.normal(0, np.pi, 4)
    params = np.array(init_arr, requires_grad = True)
    
    opt = qml.AdamOptimizer(stepsize=step_size)
    
    #opt = qml.GradientDescentOptimizer(stepsize=step_size)
    
    
    gd_param_history = [params]
    gd_cost_history = []
    def cost_fn(params):
        return model(params,H)
    for n in range(max_iterations):
    
        # Take step
        params, prev_energy = opt.step_and_cost(cost_fn, params)
        gd_param_history.append(params)
        gd_cost_history.append(prev_energy)
    
        energy = cost_fn(params)
    
        # Calculate difference between new and old energies
        conv = np.abs(energy - prev_energy)
    
        if n % 20 == 0:
            print(
                "Iteration = {:},  Energy = {:.8f} Ha,  Convergence parameter = {"
                ":.8f} Ha".format(n, energy, conv)
            )
    
        if conv <= conv_tol:
            break
    return params

# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    params = train(ins)
    return str(model(params, create_Hamiltonian(ins)))


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-1
    ), "The expected value is not correct."


test_cases = [['1.0', '-5.226251859505506'], ['2.3', '-9.66382463698038']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '1.0'...
Iteration = 0,  Energy = 3.18213619 Ha,  Convergence parameter = 0.00706463 Ha
Iteration = 20,  Energy = 2.81429406 Ha,  Convergence parameter = 0.02802447 Ha
Iteration = 40,  Energy = 2.12890891 Ha,  Convergence parameter = 0.03731618 Ha
Iteration = 60,  Energy = 1.40261584 Ha,  Convergence parameter = 0.03432551 Ha
Iteration = 80,  Energy = 0.77940152 Ha,  Convergence parameter = 0.02819345 Ha
Iteration = 100,  Energy = 0.27859865 Ha,  Convergence parameter = 0.02242495 Ha
Iteration = 120,  Energy = -0.13916448 Ha,  Convergence parameter = 0.02060997 Ha
Iteration = 140,  Energy = -0.61108691 Ha,  Convergence parameter = 0.02759354 Ha
Iteration = 160,  Energy = -1.26057921 Ha,  Convergence parameter = 0.03602257 Ha
Iteration = 180,  Energy = -1.99636795 Ha,  Convergence parameter = 0.03597526 Ha
Iteration = 200,  Energy = -2.65740451 Ha,  Convergence parameter = 0.03037369 Ha
Iteration = 220,  Energy = -3.21581670 Ha,  Convergence parameter = 0.